# Create filenames, filepaths, and JSON


In [1]:
% add bids-matlab to path
addpath(fullfile(pwd, '..'));

% TO DO: rename the bids.internal.warning function instead of silencing warnings
warning('off','all');

## Intialising a new BIDS dataset

This can be useful when you are going to output your analysis or your data acquisition into a new dataset.

In [2]:
help bids.init



 Initialize dataset with README, description, folder structure...

 init(pth, folders, is_derivative, is_datalad_ds)

 (C) Copyright 2021 BIDS-MATLAB developers


Additional help for built-in functions and operators is
available in the online version of the manual.  Use the command
'doc <topic>' to search the manual index.

Help and information about Octave is also available on the WWW
at http://www.octave.org and via the help@octave.org
mailing list.


Derivatives datasets have some extra info in their `dataset_description.json`.

If you are going to curate the dataset with Datalad, you can also mention it and this will modify the README to add extra info about this (taken from the datalad handbook).

In [8]:
is_derivative = true;
is_datalad_ds = true;

In [13]:
pth = fullfile(pwd, 'dummy_ds');

folders.subjects = {'01', '02'};
folders.sessions = {'pre', 'post'};
folders.modalities = {'anat', 'eeg'};

In [14]:
bids.init(pth, folders, is_derivative, is_datalad_ds)

In [15]:
!tree dummy_ds

dummy_ds
├── 01
│   ├── post
│   │   ├── anat
│   │   └── eeg
│   └── pre
│       ├── anat
│       └── eeg
├── 02
│   ├── post
│   │   ├── anat
│   │   └── eeg
│   └── pre
│       ├── anat
│       └── eeg
├── CHANGES
├── dataset_description.json
└── README

14 directories, 3 files



Template README was generated.

In [16]:
!cat dummy_ds/README

# README

The README is usually the starting point for researchers using your data
and serves as a guidepost for users of your data. A clear and informative
README makes your data much more usable.

In general you can include information in the README that is not captured by some other
files in the BIDS dataset (dataset_description.json, events.tsv, ...).

It can also be useful to also include information that might already be 
present in another file of the dataset but might be important for users to be aware of 
before preprocessing or analysing the data.

If the README gets too long you have the possibility to create a `/doc` folder
and add it to the `.bidsignore` file to make sure it is ignored by the BIDS validator.

More info here: https://neurostars.org/t/where-in-a-bids-dataset-should-i-put-notes-about-individual-mri-acqusitions/17315/3

## Details related to access to the data

- [ ] Data user agreement

If the dataset requires a data user agreement, link to the relevant infor

In [17]:
!cat dummy_ds/dataset_description.json

{
    "Name": "",
    "BIDSVersion": "",
    "DatasetType": "derivative",
    "License": "",
    "Authors": "",
    "Acknowledgements": "",
    "HowToAcknowledge": "",
    "Funding": "",
    "ReferencesAndLinks": "",
    "DatasetDOI": "",
    "HEDVersion": "",
    "GeneratedBy": [
        {
            "Name": "",
            "Version": "",
            "Description": "",
            "CodeURL": "",
            "Container": {
                "Type": "",
                "Tag": ""
            }
        }
    ],
    "SourceDatasets": [
        {
            "DOI": "",
            "URL": "",
            "Version": ""
        }
    ]
}


 ## Generating filenames

Uses the `bids.create_filename` function.

In [42]:
help bids.create_filename



 Creates a BIDS compatible filename and can be used to create new names to rename files

 USAGE::

   [filename, pth, json] = bids.create_filename(p)

 :param p:  specification of the filename to create, very similar to the output of
                   ``bids.internal.parse_filename``
 :type  p:  structure

 Content of ``p``:

   - ``p.suffix``        - required
   - ``p.ext``           - extension (default: ``p.ext = ''``)
   - ``p.entities``      - structure listing the entity-label pairs to compose the filename
   - ``p.prefix``        - prefex to prepend (default: ``p.prefix = ''``)
   - ``p.use_schema``    - bollean to check required entities for a given suffix,
                           and reorder entities according to the BIDS schema.
   - ``p.entity_order``  - user specified order in which to arranges the entities
                           in the filename. Overrides ``p.use_schema``.

 If no entity order is specified and the filename creation is not based on the BIDS
 sche

In [28]:
p.suffix = 'bold';
p.ext = '.nii';
p.entities = struct( ...
                  'sub', '01', ...
                  'ses', 'test', ...
                  'task', 'face recognition', ...
                  'run', '02');

[filename, pth] = bids.create_filename(p)

filename = sub-01_ses-test_task-faceRecognition_run-02_bold.nii
pth = sub-01/ses-test/func


By default this relies on the BIDS schema to know in which order the entities must go for a certain `suffix` type. This can also tell you if you are missing a required entity.

In [40]:
p.suffix = 'bold';
p.ext = '.nii';
p.entities = struct( ...
                  'sub', '01', ...
                  'ses', 'test', ...
                  'run', '02');

[filename, pth] = bids.create_filename(p)

error: The entity task cannot not be empty for the suffix bold
error: called from
    create_filename at line 69 column 7


This behavior can be turned off with the `use_schema` "flag".

In [41]:
p.suffix = 'bold';
p.ext = '.nii';
p.use_schema = false;
p.entities = struct( ...
                  'sub', '01', ...
                  'ses', 'test', ...
                  'run', '02');

[filename, pth] = bids.create_filename(p)

filename = sub-01_ses-test_run-02_bold.nii
pth = sub-01/ses-test/func


Or you can specify the order of the entities manually.

In [43]:
p.suffix = 'bold';
p.ext = '.nii';
p.entity_order = {'run', 'sub', 'ses'};
p.entities = struct( ...
                  'sub', '01', ...
                  'ses', 'test', ...
                  'run', '02');

[filename, pth] = bids.create_filename(p)

filename = run-02_sub-01_ses-test_bold.nii
pth = sub-01/ses-test/func


## Modifying filenames

This can be used to modify the entities.

In [45]:
clear p
filename = 'sub-01_ses-test_task-faceRecognition_run-02_bold.nii'
p.entities = struct( ...
                  'sub', '02', ...
                  'task', 'new task');

[filename, pth] = bids.create_filename(p, filename)

filename = sub-01_ses-test_task-faceRecognition_run-02_bold.nii
filename = sub-02_ses-test_task-newTask_run-02_bold.nii
pth = sub-02/ses-test/func


An entity can also be removed from the filename.

In [25]:
p.entities = struct('ses', '');
filename = bids.create_filename(p, filename)

filename = sub-02_task-newTask_run-02_bold.nii


This can also be useful to remove the prefix of some files.

In [35]:
filename = 'wuasub-01_ses-test_task-faceRecognition_run-02_bold.nii';

p.prefix = '';

filename = bids.create_filename(p, filename)

filename = sub-01_ses-test_task-faceRecognition_run-02_bold.nii


## Generating file names for derivatives

This can prove useful as this function will also provide youi with a dummy json that should accompany derivatives files.

In [49]:
filename = 'sub-01_ses-test_task-faceRecognition_run-02_bold.nii';

p.entities = struct('desc', 'preproc');
p.use_schema = false;

[filename, pth, json] = bids.create_filename(p, filename)

filename = sub-01_ses-test_task-faceRecognition_run-02_desc-preproc_bold.nii
pth = sub-01/ses-test/func
json =

  scalar structure containing the fields:

    filename = sub-01_ses-test_task-faceRecognition_run-02_desc-preproc_bold.json
    content =

      scalar structure containing the fields:

        Description = RECOMMENDED
        Sources = 
        {
          [1,1] = 
          {
            [1,1] = OPTIONAL
          }
        }
        RawSources = 
        {
          [1,1] = 
          {
            [1,1] = OPTIONAL
          }
        }
        SpatialReference = 
        {
          [1,1] = 
          {
            [1,1] = REQUIRED if no space entityor if non standard space RECOMMENDED otherwise
          }
        }




The content of the JSON should adapt depending on the entities or suffix present in the output filename.

In [51]:
filename = 'sub-01_ses-test__res-hi_desc-brain_mask.nii';

p.entities = struct('desc', 'preproc');
p.use_schema = false;

[filename, pth, json] = bids.create_filename(p, filename)

filename = sub-01_ses-test_res-hi_desc-preproc_mask.nii
pth = sub-01/ses-test
json =

  scalar structure containing the fields:

    filename = sub-01_ses-test_res-hi_desc-preproc_mask.json
    content =

      scalar structure containing the fields:

        Description = RECOMMENDED
        Sources = 
        {
          [1,1] = 
          {
            [1,1] = OPTIONAL
          }
        }
        RawSources = 
        {
          [1,1] = 
          {
            [1,1] = REQUIRED
          }
        }
        SpatialReference = 
        {
          [1,1] = 
          {
            [1,1] = REQUIRED if no space entityor if non standard space RECOMMENDED otherwise
          }
        }
        Resolution = 
        {
          [1,1] = 
          {
            [1,1] =

              scalar structure containing the fields:

                hi: 1x24 sq_string

          }
        }
        Atlas = 
        {
          [1,1] = 
          {
            [1,1] = OPTIONAL
          }
        